In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Chicken Disease Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\Chicken Disease Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    unzip_dir : Path
    local_data_file : Path


In [67]:
from CDC.constants import *
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')

In [68]:
from CDC.utils.helper_functions import read_yaml, create_directories

In [69]:

from box import ConfigBox

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> ConfigBox:
        config = self.config.data_ingestion
        create_directories([config.root_dir])    

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            unzip_dir = config.unzip_dir,
            local_data_file = config.local_data_file,
            source_url = config.source_URL
        )

        return data_ingestion_config

In [70]:
import os
import urllib.request as request
import zipfile
from CDC import logger

In [71]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")  

    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [72]:
try:
    configuration_manager = ConfigurationManager()
    configuration = configuration_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(configuration)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    print(e)

[2023-06-20 23:41:12,455: INFO: helper_functions: yaml file {filename} uploaded successfully]
[2023-06-20 23:41:12,459: INFO: helper_functions: yaml file {filename} uploaded successfully]
[2023-06-20 23:41:12,461: INFO: helper_functions: created directory at: artifacts]
[2023-06-20 23:41:12,465: INFO: helper_functions: created directory at: artifacts/data]
[2023-06-20 23:41:34,618: INFO: 1532638196: artifacts/data/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 856A:13A7:125B61:15F34C:6491F269
Accept-Ranges: bytes
Date: Tue, 20 Jun 2023 18:41:13 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990028-F